In [6]:
#from app.db import connect
import pandas as pd
import numpy as np
import psycopg2

df=pd.read_csv('myCSV/names4.csv') 
df.index = np.arange(6001, len(df) + 6001)
data = pd.DataFrame(df)
data.to_csv('myCSV/names4_1.csv', index_label='id', header=False)

In [13]:
from app import cfi
from app.db import connectWrite
import pandas as pd
from sqlalchemy import bindparam, text, String
from io import StringIO
from psycopg2 import sql
import psycopg2 as pg

con = connectWrite()
statement = "CREATE TABLE IF NOT EXISTS temporary.source of datasetcustomers;"  
con.execute(statement)
statement = "TRUNCATE TABLE temporary.source;"  
con.execute(statement)
con.close()
con = cfi
buffer = StringIO()
data.to_csv(buffer, index_label='id', header=False)
buffer.seek(0)
con = pg.connect(**cfi)    
cursor = con.cursor()
try: 
    string = sql.SQL(        
        """
        copy temporary.source
        from stdin (
            format csv,
            null "NULL",
            delimiter ','
        );
        """)
    #with open(fln) as csv_file: 
    cursor.copy_expert(string, buffer)
    con.commit()
except (Exception, psycopg2.DatabaseError) as error: 
    print("Error: %s" % error)
    con.rollback()
    cursor.close()
    #return 1
print("copy_from_stringio() done")
cursor.close()
con.close()


Connecting...
copy_from_stringio() done


In [15]:
con = connectWrite()
stmt = "CALL test_run();"
con.execute(stmt)
con.close()
print(data)

Connecting...
      firstname  lastname
6001      Alyda     Izaak
6002  Blondelle    Daegal
6003    Shirlee     Astra
6004   Dulcinea     Nunci
6005    Kirstin   Colleen
...         ...       ...
7996    Imojean    Irmine
7997   Cindelyn     Levey
7998      Eadie    Daegal
7999      Diena     Rubie
8000    Lisette  Magnolia

[2000 rows x 2 columns]
